In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [3]:
!pip install transformers
!pip install keras

     |████████████████████████████████| 665kB 4.6MB/s 
     |████████████████████████████████| 3.8MB 11.6MB/s 
     |████████████████████████████████| 890kB 41.1MB/s 
     |████████████████████████████████| 1.1MB 52.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=266e6e31cbea0bf435f96fccecdd15056d10783b76ee5867679be363b95c6e41
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
%matplotlib inline
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras.preprocessing.sequence import pad_sequences

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


Using TensorFlow backend.


In [8]:
df = pd.read_csv("https://raw.githubusercontent.com/liu431/Content-Analysis-Project/master/Data/output_review.csv")
df_meta = pd.read_csv("https://raw.githubusercontent.com/liu431/Content-Analysis-Project/master/Data/output_meta.csv")
df.head()

,Unnamed: 0,asin,title,overall,vote,verified,reviewTime,reviewerID,style,reviewerName,reviewText,summary,unixReviewTime,image
0,0,B000A6PPOK,Microsoft Natural Ergonomic Keyboard 4000,5.0,"1,113",True,"10 18, 2005",AILCWT1IIP7ZT,{'Style:': ' Retail'},Charles Chen,"As a software developer, I am literally attach...",Best Overall Keyboard to Date,1129593600,NaN
1,1,B000A6PPOK,Microsoft Natural Ergonomic Keyboard 4000,5.0,16,True,"10 12, 2005",A300T403J8526F,{'Style:': ' Retail'},T. Becker,"I've had a Natural Keyboard Pro for years, and...",Best keyboard I've used,1129075200,NaN
2,2,B000A6PPOK,Microsoft Natural Ergonomic Keyboard 4000,4.0,11,True,"10 5, 2005",AFPGV3IQ9K691,{'Style:': ' Retail'},GDC,"As with most Microsoft products, great hardwar...","Great keyboard, poor software.",1128470400,NaN
3,3,B000A6PPOK,Microsoft Natural Ergonomic Keyboard 4000,5.0,NaN,True,"05 15, 2018",A2FMMCFQ8702DN,{'Style:': ' Business'},Wickenball,Love it.,Five Stars,1526342400,NaN
4,4,B000A6PPOK,Microsoft Natural Ergonomic Keyboard 4000,5.0,NaN,True,"05 12, 2018",A1I8WBJSZIPJE0,{'Style:': ' Business'},B. Stearn,I've had various versions of this keyboard for...,Five Stars,1526083200,NaN


In [9]:
df = df[['asin', 'reviewText']]
df.head()

,asin,reviewText
0,B000A6PPOK,"As a software developer, I am literally attach..."
1,B000A6PPOK,"I've had a Natural Keyboard Pro for years, and..."
2,B000A6PPOK,"As with most Microsoft products, great hardwar..."
3,B000A6PPOK,Love it.
4,B000A6PPOK,I've had various versions of this keyboard for...


Deep Learning & Text Generation

In [10]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer_gpt = AutoTokenizer.from_pretrained("gpt2")
model_gpt = AutoModelWithLMHead.from_pretrained("gpt2")

In [0]:
from sklearn.model_selection import train_test_split
train_text, test_text = train_test_split(df['reviewText'], test_size=0.2)

In [0]:
train_text.to_frame().to_csv(r'train_text', header=None, index=None, sep=' ', mode='a')
test_text.to_frame().to_csv(r'test_text', header=None, index=None, sep=' ', mode='a')

In [13]:
train_text.head()

1818    I have used a MS board for 7 years which is a ...
5403    ... and she seems to love the size of the keys...
5033    The keys stick like crazy!  Or just don't pres...
5362    I bought this because I often end up typing at...
6113                               Works fine nice remote
Name: reviewText, dtype: object

In [0]:
torch.cuda.empty_cache() 

In [15]:
!python run_language_modelling.py --output_dir=output_gpt_text --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=train_text --do_eval --eval_data_file=test_text --per_gpu_train_batch_size=1 --per_gpu_eval_batch_size=1

2020-06-01 04:50:57.535318: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
06/01/2020 04:50:59 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
06/01/2020 04:50:59 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /root/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.db13c9bc9c7bdd738ec89e069621d88e05dc670366092d809a9cbcac6798e24e
06/01/2020 04:50:59 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_e

In [27]:
!python run_generation.py --model_type=gpt2 --model_name_or_path=output_gpt_text

2020-06-01 05:10:34.417109: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
06/01/2020 05:10:36 - INFO - transformers.tokenization_utils -   Model name 'output_gpt_text' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'output_gpt_text' is a path, a model identifier, or url to a directory containing tokenizer files.
06/01/2020 05:10:36 - INFO - transformers.tokenization_utils -   Didn't find file output_gpt_text/added_tokens.json. We won't load it.
06/01/2020 05:10:36 - INFO - transformers.tokenization_utils -   loading file output_gpt_text/vocab.json
06/01/2020 05:10:36 - INFO - transformers.tokenization_utils -   loading file output_gpt_text/merges.txt
06/01/2020 05:10:36 - INFO - transformers.tokenization_utils -   loading file None
06/01/2020 05:10:36 - INFO - transformers.tokenization_utils -   loading file output_gpt_text/special_tokens_map.json
06/01/2020 05:

#### Text Generation Result: This keyboard has keyboard ports for my Mac Pro. The left hand side is on the middle row, so I can use my computer on my second setup.  In my previous setup the split keyboard was left on the right for both sides.  I switched to one right hand to keep my keyboard (left) with my iPad and keyboard tray (right) this time (right) and now my desktops are on the right for a different keyboard tray. If you are new to using a split keyboard, this will be a nice option.